### 0. Install the package
You need to install the abstcal package first. If you've done this already, you can skip this step.

In [ ]:
!pip install abstcal
# the test version
# !pip install -i https://test.pypi.org/simple/ abstcal-ycui1

### 1. Import the package
Before you use this package, please make sure that your datasets have been pre-processed
to have the correct format. You can find more information about the required format of the datasets
on the package's GitHub page.

In [ ]:
from abstcal import TLFBData, VisitData, AbstinenceCalculator

### 2. Prepare the visit dataset


In [ ]:
# Import the visit dataset by specifying the path to the visit dataset
test_visit_filepath = "https://github.com/ycui1/abstcal/blob/dfac952bd85ac0303ec05c8578222684a600a4f8/test_visit.csv"
visit_data = VisitData(test_visit_filepath)

In [ ]:
# Profile the visit data
visit_data.profile_data()

In [ ]:
# Calculate the retention rates (optional)
visit_data.get_retention_rates()

In [ ]:
# Check if any subjects have their visit dates out of the correct order
visit_data.get_out_of_order_visit_data()

In [ ]:
# Remove any records with missing id, visit, or date
visit_data.drop_na_records()

In [ ]:
# Impute the visit dataset by filling the missing dates. By default, it will impute the
# missing dates using the most frequent interval since the reference date
visit_data.impute_data()

### 3. Prepare the biochemical verification measures dataset

In [ ]:
# Read the CO data, by specifying the abstinence cutoff, which will be used in later processing steps
test_co_filepath = "https://github.com/ycui1/abstcal/blob/dfac952bd85ac0303ec05c8578222684a600a4f8/test_co.csv"
co_data = TLFBData(test_co_filepath, abst_cutoff=4)

In [ ]:
# Profile the CO data
co_data.profile_data()

In [ ]:
# Interpolate the biochemical data by specifying the half life in days and the number
# of days for interpolation preceding the current measurement day
co_data.interpolate_biochemical_data(0.25, 1)

In [ ]:
# After the interpolation, you can profile the CO data again if needed
co_data.profile_data()

In [ ]:
# Remove any records with missing id, date, or amount
co_data.drop_na_records()

In [ ]:
# Check and remove any duplicates
co_data.check_duplicates()

### 4. Prepare the TLFB dataset

In [ ]:
# Read the TLFB dataset, by default, the abstinence cutoff is zero usage
test_tlfb_filepath = "https://github.com/ycui1/abstcal/blob/dfac952bd85ac0303ec05c8578222684a600a4f8/test_tlfb.csv"
tlfb_data = TLFBData(test_tlfb_filepath)

In [ ]:
# Profile the TLFB data
tlfb_data.profile_data()

In [ ]:
# Drop any records with missing id, date, or amount
tlfb_data.drop_na_records()

In [ ]:
# Check and remove any duplicates
tlfb_data.check_duplicates()

In [ ]:
# Recode the TLFB data to remove any outliers
tlfb_data.recode_data()

In [ ]:
# Impute the TLFB data, an important while optional argument is the biochemical data
# It will help invalidate false negatives
tlfb_data.impute_data(biochemical_data=co_data)

### 5. Calculate Abstinence
After you have cleaned up the TLFB and visit data, you're ready to calculate abstinence.


In [ ]:
# Create the calculator by passing the TLFB and visit data
calculator = AbstinenceCalculator(tlfb_data, visit_data)

In [ ]:
# Calculate the 7-day point-prevalence for visits 3 and 4
abst_pp7, lapses_pp7 = calculator.abstinence_pp([3, 4], 7)

In [ ]:
# Calculate the continuous abstinence for visits 3 and 4
abst_cont, lapses_cont = calculator.abstinence_cont(1, [3, 4])

In [ ]:
# Calculate the prolonged abstinence without lapses for visits 3 and 4
abst_prol, lapses_prol = calculator.abstinence_prolonged(1, [3, 4], False)

In [ ]:
# Calculate the prolonged abstinence with lapses (> 5 cigs) for visits 3 and 4
abst_pros, lapses_pros = calculator.abstinence_prolonged(1, [3, 4], "5 cigs")

### 6. Merge and export data

In [ ]:
# Replace the target_filepath to where you want to save your merged datafile
abst_filepath = "merged_abst.csv"
abst_merged = calculator.merge_abst_data([abst_pp7, abst_cont, abst_prol, abst_pros], abst_filepath)

In [ ]:
lapses_filepath = "merged_lapses.csv"
lapses_merged = calculator.merge_lapse_data([lapses_pp7, lapses_cont, lapses_prol, lapses_pros], lapses_filepath)